At the end of this notebook, we will have submitted to the ["Paddy Doctor: Paddy Disease Classification" competition on Kaggle](https://www.kaggle.com/competitions/paddy-disease-classification/overview)!

Let us begin by downloading the data.

In [13]:
%%bash

exec bash
# rm -rf data
# mkdir data
apt install unzip


Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [1]:
pip install -U timm

Note: you may need to restart the kernel to use updated packages.


In [15]:
!cd data && kaggle competitions download -c paddy-disease-classification && unzip -q paddy-disease-classification.zip

paddy-disease-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import timm

We have now downloaded and extracted the data to the `data` directory.

In [3]:
ls data

paddy-disease-classification.zip  submissions/  train.csv
sample_submission.csv             test_images/  train_images/


In [4]:
ls data/train_images

bacterial_leaf_blight/     blast/                       dead_heart/    normal/
bacterial_leaf_streak/     brown_spot/                  downy_mildew/  tungro/
bacterial_panicle_blight/  convnext_large_in22k_squish  hispa/


In [5]:
ls data/test_images | head

200001.jpg
200002.jpg
200003.jpg
200004.jpg
200005.jpg
200006.jpg
200007.jpg
200008.jpg
200009.jpg
200010.jpg
ls: write error: Broken pipe


Seems that the train data is organized by directories, with the name of the directory being the label.

Test images just live in `data/test_images`

Let us see what is the format of the sample submission file to have a full picture.

In [6]:
import pandas as pd

sample_sub = pd.read_csv('data/sample_submission.csv')
sample_sub.head()

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN


Mhmm. Guessing the labels for the submission are the names of the directories.

Ok, let's start training!

In [7]:
from fastai.vision.all import *
from fastcore.parallel import *

In [8]:
path = Path('data')
trn_path= path/'train_images'
tst_files = get_image_files(path/'test_images').sorted()

In [9]:
tta_res = []

In [10]:
Path()

Path('.')

In [11]:
#def train(desc, arch, item, batch, accum=False):
#    kwargs = {'bs':32} if accum else{}
#    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item, batch_tfms=batch, **kwargs)
#    cbs = GradientAccumulation(2) if accum else []
#    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
#    learn.fine_tune(12, 0.01)
#    tta_res.append(learn.tta(dl=dls.test_dl(tst_files)))
#    learn.export(f'{arch}_{desc}')

In [12]:
arch = 'convnext_large_in22k'

In [13]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, 
   item_tfms=Resize(480, method='squish'), batch_tfms=aug_transforms(size=224, min_scale=0.75), bs=32, accum=True)

tst_dl = dls.test_dl(tst_files)

In [14]:
learn = vision_learner(dls, arch, metrics=error_rate).to_fp16()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.788541,0.522027,0.170111,05:53


In [ ]:
learn = vision_learner(dls, arch, metrics=error_rate).to_fp16()
learn.fine_tune(12, 0.01)
learn.export('cn_sml_12')

In [23]:
test_dl = learn.dls.test_dl(get_image_files('data/test_images'))

In [24]:
test_preds = learn.get_preds(dl=test_dl)

In [25]:
sample_sub

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN
...,...,...
3464,203465.jpg,NaN
3465,203466.jpg,NaN
3466,203467.jpg,NaN
3467,203468.jpg,NaN


In [26]:
get_image_files('data/test_images')

(#3469) [Path('data/test_images/200770.jpg'),Path('data/test_images/203307.jpg'),Path('data/test_images/203265.jpg'),Path('data/test_images/201796.jpg'),Path('data/test_images/202973.jpg'),Path('data/test_images/201780.jpg'),Path('data/test_images/201052.jpg'),Path('data/test_images/201179.jpg'),Path('data/test_images/201783.jpg'),Path('data/test_images/200080.jpg')...]

In [27]:
predicted_classes = [learn.dls.train_ds.vocab[i] for i in test_preds[0].argmax(1)]

In [28]:
predicted_classes[:10]

['bacterial_leaf_streak',
 'blast',
 'dead_heart',
 'bacterial_leaf_blight',
 'downy_mildew',
 'hispa',
 'brown_spot',
 'tungro',
 'normal',
 'dead_heart']

And the last thing we need to output predictions are file names.

In [29]:
fns = [p.name for p in get_image_files('data/test_images')]

In [30]:
sub = pd.DataFrame(data={'image_id': fns, 'label': predicted_classes})

In [ ]:
# mkdir data/submissions

In [31]:
sub.to_csv('data/submissions/brismith_sub2_vit.csv.gz', index=False)

In [32]:
!kaggle competitions submit -c paddy-disease-classification -f data/submissions/brismith_sub1.csv.gz -m "2nd sub vit walkthru11"

Traceback (most recent call last):
  File "/root/.local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/root/.local/lib/python3.7/site-packages/kaggle/cli.py", line 67, in main
    out = args.func(**command_args)
  File "/root/.local/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 562, in competition_submit_cli
    competition, quiet)
  File "/root/.local/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 513, in competition_submit
    content_length=os.path.getsize(file_name),
  File "/opt/conda/lib/python3.7/genericpath.py", line 50, in getsize
    return os.stat(filename).st_size
FileNotFoundError: [Errno 2] No such file or directory: 'data/submissions/brismith_sub1.csv.gz'


In [33]:
!kaggle competitions submit -c paddy-disease-classification -f data/submissions/brismith_sub2_vit.csv.gz -m "2nd sub vit walkthru11"

100%|██████████████████████████████████████| 11.7k/11.7k [00:00<00:00, 24.9kB/s]
Successfully submitted to Paddy Doctor: Paddy Disease Classification